<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex10notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex10notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo10.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
# SciPy の 統計関数群の中の正規分布モジュール (scipy.stats.norm) と多変量正規分布モジュール (scipy.stats.multivariate_normal)
from scipy.stats import norm, multivariate_normal

---
## 判別分析 (1)
---



---
### 判別分析とは



#### 例： $(\mbox{身長}, \mbox{体重})$ から「人間」か「ほげ星人」かを判別する

「人間」と「ほげ星人」あわせて200人分の $(\mbox{身長[cm]}, \mbox{体重[kg]})$ の数値を集めたデータがあったとします．それぞれのデータには，人間の数値なのかほげ星人の数値なのかを表す変数も含まれているとします（注）．
以下のセルの出力の `Height` 列は身長，`Weight` 列は体重を表します．`Class` 列の値は，人間を表す `Human` とほげ星人を表す `Hoge` のいずれかです．

<span style="font-size: 75%">
※注: これは，説明を理解してもらいやすくするために作った嘘のデータです．このデータ中のほげ星人に近い身長・体重のひとがいたとしても，そのひとが人間ではないというようなことを主張するものではありません．
</span>

In [ ]:
# 人間 vs ほげ星人
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/humanvshoge.csv'
dfHoge = pd.read_csv(URL)
dfHoge

`Class` の値で色分けして (`Height`, `Weight`) の散布図を描いてみると，次のようになっています．

In [ ]:
# 「人間 vs ほげ星人」の散布図
fig, ax = plt.subplots(figsize=(8, 6))
dfTmp = dfHoge.loc[dfHoge['Class'] == 'Human']
ax.scatter(dfTmp['Height'], dfTmp['Weight'], label='Human')
dfTmp = dfHoge.loc[dfHoge['Class'] == 'Hoge']
ax.scatter(dfTmp['Height'], dfTmp['Weight'], label='Hoge')
xmin, xmax = 0, 250
ymin, ymax = 0, 150
ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.set_ylim(ymin, ymax)
ax.set_ylabel('Weight')
ax.set_aspect('equal')
ax.legend()
plt.show()

このようなデータが与えられたときに，「人間かほげ星人か分からない（`Class` の値が未知な）ひとの (`Height`, `Width`) の値から，そのひとの `Class` を決定したい」というのが，判別分析の問題の一つの例です．

#### 問題設定

上記の例では，個々のデータが「人間」と「ほげ星人」のどちらのグループに属するのか決めることを考えていました．判別分析では，これらのグループのことを，**クラス** (class) や **カテゴリ** (category) と呼ぶことがあります．この例では「人間」クラスと「ほげ星人」クラスの二つを考えていますが，クラスの数は3つ以上あっても構いません．

判別分析でやりたいことを一般化して説明すると，次のようになります：

> 多変量のデータがあり，その個々のデータは複数のクラスのうちのいずれかに所属しているとする．これらのデータを基にして，新しいデータがどのクラスに所属するのかを決定する仕組み（これを **判別関数** といいます）を作りたい．

統計学ではこのような問題を **判別分析** (discriminant analysis) と呼びますが，機械学習等の分野では，**分類** (classification) や **識別** (discrimination)，あるいは **パターン認識** (pattern recognition) と呼びます（「機械学習I/II」の授業で出てきます）．


これまで扱ってきた多変量解析手法では，変数はすべて量的な変数でしたが，判別分析では，説明変数が量的な変数で，被説明変数は質的な変数（注）となっているとみなせます．

<span style="font-size: 75%">
※ 注意: より詳しく言うと，「人間」と「ほげ星人」みたいに少数の値をとる，「カテゴリカル」な変数．
</span>


#### 例： Fisher のアヤメのデータ

判別分析の問題の例をもう一つあげます．植物の [アヤメ](https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%A4%E3%83%A1) の種を分類する問題です．

In [ ]:
# Fisher のアヤメのデータ
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
dfIris = iris.frame
dfIris['species'] = 'hoge'
for k, tn in enumerate(iris.target_names):
    dfIris.loc[dfIris['target'] == k, 'species'] = tn
dfIris.drop(columns='target', inplace=True)
dfIris

このデータは，

- Iris setosa: [Wikipedia](https://en.wikipedia.org/wiki/Iris_setosa)，[Wikipedia日本語版 ヒオウギアヤメ]( https://ja.wikipedia.org/wiki/%E3%83%92%E3%82%AA%E3%82%A6%E3%82%AE%E3%82%A2%E3%83%A4%E3%83%A1)
- Iris versicolor [Wikipedia](https://en.wikipedia.org/wiki/Iris_versicolor)
- Iris virginica [Wikipedia](https://en.wikipedia.org/wiki/Iris_virginica)

という3種のアヤメの`sepal length`（がく片の長さ[cm]），`sepal width`（がく片の幅[cm]），`petal length`（花弁の長さ[cm]），`petal width`（花弁の幅[cm]）の数値から成るものです．

4つの数値が組になったデータですので，先の例のようにそのまま散布図を描くことはできませんが，4つのうち任意の2つの変数を選んで散布図を描くと，次のようになっています．

In [ ]:
# Seaborn のペアプロット関数（多次元データの変数間の散布図等を自動的に描いてくれる）
seaborn.pairplot(dfIris, hue='species')

このデータに対する判別分析の問題は，
何らかの方法で，(`sepal length`, `sepal width` , `petal length`, `petal width`) という4つの数値から，この数値の組を持つアヤメの `species` （setosa / versicolor / virginica のいずれか）を予測する，ということになります．

余談ですが，このアヤメのデータは，有名な統計学者である Ronald Fisher が判別分析の研究に用いたことから，「Fisher's Iris data set」として知られています．多変量解析や機械学習・パターン認識の教科書等によく出てきます．ここでは，scikit-learn という機械学習ライブラリで提供されているものを利用しています．

- Ronald Fisher （ロナルド・フィッシャー）:  イギリスの統計学者・進化生物学者， 1890 - 1962， [Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%AD%E3%83%8A%E3%83%AB%E3%83%89%E3%83%BB%E3%83%95%E3%82%A3%E3%83%83%E3%82%B7%E3%83%A3%E3%83%BC)
- [sklearn.datasets.load_iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)

---
### 2クラス1次元の判別分析

ここでは，データは1次元，つまり1つの変数のみから成るものとし，さらにクラスの数を 2 に限定して，判別分析の考え方や具体的な方法を説明します．

#### 考え方




判別分析では，データの背後に正規分布を仮定します．つまり，個々のデータは，何らかの決まったパラメータを持った正規分布から抽出された標本だとします．その正規分布は，クラスごとに一つずつ考えます．例えば，1次元のデータを「クラス1」と「クラス2」の2クラスに分類する問題の場合，

- クラス1の正規分布 ${\cal N}(\mu_1, \sigma_1^2)$
- クラス2の正規分布 ${\cal N}(\mu_2, \sigma_2^2)$

という2つを考えます．これらの正規分布のパラメータは，与えられたデータから適当な方法で推定します．

このとき，「$x$ という値をもつデータがクラス1の正規分布から得られたとすることの尤度」と，「$x$ という値をもつデータがクラス2の正規分布から得られたとすることの尤度」を求めてやれば，「$x$ という値をもつデータは，尤度のより大きい方の正規分布に対応するクラスに属すと判定する」ことができるでしょう．これが，判別分析の基本的な考え方です．

上記の2つの正規分布の場合，尤度はぞれぞれ
$$
\begin{aligned}
\ell_1(x) &= \frac{1}{\sqrt{2\pi\sigma_1^2}} \exp{\left( - \frac{(x-\mu_1)^2}{2\sigma_1^2}\right)} \qquad (1)\\
\ell_2(x) &= \frac{1}{\sqrt{2\pi\sigma_2^2}} \exp{\left( - \frac{(x-\mu_2)^2}{2\sigma_2^2}\right)} \qquad (2)\\
\end{aligned}
$$
と表せます．この $\ell_1(x)$ と $\ell_2(x)$ の値を求めて，「
$\ell_1(x) \geq \ell_2(x)$ ならクラス1，さもなくばクラス2に属すと判定する」ことになります．

下図において，青い方がクラス1の，オレンジの方がクラス2の正規分布の確率密度関数だったとすると，次のようになります．

- $\ell_1(x_1) > \ell_2(x_1)$ なので，$x = x_1$ のデータはクラス1に分類
- $\ell_1(x_2) < \ell_2(x_2)$ なので，$x = x_2$ のデータはクラス2に分類




<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/likelihood.png">

#### マハラノビス距離と線形判別関数

上記の説明では，2クラスの正規分布は平均も分散も任意としています．しかし，これにもう少し制約を加えると，判別のための計算が（判別関数が）より簡単な式になります．具体的には，「2クラスの正規分布は分散が等しい」，つまり，$\sigma_1^2 = \sigma_2^2 = \sigma^2$ と表せると仮定します．

このときは，対数尤度 $\ell_1(x), \ell_2(x)$ のかわりに次の量を求めて大小を比較することで，対数尤度を用いたときと等価な判別ができます（注）．


$$
\begin{aligned}
D_1^2(x) &= \frac{(x-\mu_1)^2}{\sigma^2} \qquad (3)\\
D_2^2(x) &= \frac{(x-\mu_2)^2}{\sigma^2} \qquad (4)
\end{aligned}
$$

ただし，負号がなくなったので大小関係は逆になることに注意．$D_1^2(x) \leq D_2^2(x)$ ならクラス1，さもなくばクラス2とすることになります．

※ 注意: この場合，$\frac{1}{\sigma^2}$ もなくしても構わないのですが，2次元以上の場合の説明とあわせる（＆↓のマハラノビス距離の説明をする）ためにあえて残しています．

ここで出てくる

$$
D(x) = \sqrt{\frac{(x-\mu)^2}{\sigma^2}} \qquad (5)
$$

のことを，**マハラノビス距離** (Mahalanobis' distance) といいます（式$(3),(4)$はマハラノビス距離の2乗）．ちゃんとした説明は省略しますが，これは，「$x$ から ${\cal N}(\mu,\sigma^2)$ までの距離」を表す量と考えることができます．

余談ですが，マハラノビスは統計学者の名前です．

Prasanta Chandra Mahalanobis（プラサンタ・チャンドラ・マハラノビス）：  インドの統計学者， 1893 - 1972，[Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%A9%E3%82%B5%E3%83%B3%E3%82%BF%E3%83%BB%E3%83%81%E3%83%A3%E3%83%B3%E3%83%89%E3%83%A9%E3%83%BB%E3%83%9E%E3%83%8F%E3%83%A9%E3%83%8E%E3%83%93%E3%82%B9)

式 $(3),(4)$ を用いて次式で定義される関数 $z(x)$ を，**線形判別関数** といいます：

$$
z(x) = \frac{D_2^2(x) - D_1^2(x)}{2} \qquad (6)
$$

あるデータ $x$ について $z(x) \ge 0$ ならばこのデータはクラス1に属すると判定し，$z(x) < 0$ ならばクラス2に属すると判定します．

この式は，次のように変形していくと，$x$ の一次式となっています．これが，「線形」判別関数と呼ばれる所以です．

$$
\begin{aligned}
z(x) &= \frac{1}{2\sigma^2}\left( (x-\mu_2)^2 - (x-\mu_1)^2 \right) \\
&= \frac{1}{2\sigma^2}\left( x^2 - 2\mu_2 x + \mu_2^2 - (x^2 - 2\mu_1 x + \mu_1^2) \right)\\
&= \frac{1}{2\sigma^2}\left( 2(\mu_1 - \mu_2)x - (\mu_1^2 - \mu_2^2) \right) \\
&= \frac{\mu_1 - \mu_2}{2\sigma^2}\left( 2x - (\mu_1 + \mu_2)\right) \\
&= \frac{\mu_1 - \mu_2}{\sigma^2}\left( x - \frac{\mu_1 + \mu_2}{2}\right) & (7)
\end{aligned}
$$

$z(x) = 0 \Leftrightarrow x = \frac{\mu_1+\mu_2}{2}$ ですので，2つの平均の中点がクラス1とクラス2を分ける境界となることが分かります．



判別分析の手法としては，判別関数が線形ではないものもいろいろ考えられるので，線形である場合の判別分析をそうでない場合と区別して，**線形判別分析** (Linear Discriminant Analysis, LDA) と呼ぶこともあります．Ronald Fisher が最初に研究したと考えられており，*Fisher's Linear Discriminant* と呼ぶこともあります．

#### 「人間」vs「ほげ星人」の判別分析

実際の例として，「人間」vs「ほげ星人」で，身長の数値のみから人間かほげ星人か判別する問題を考えてみましょう．

まず，「人間」と「ほげ星人」それぞれの身長の平均 $\mu_1, \mu_2$ を推定します（ここでは標本平均を用いています）．



In [ ]:
# X_Human が「人間」の身長，X_Hoge が「ほげ星人」の身長
X_Human = dfHoge.loc[dfHoge['Class'] == 'Human', 'Height'].to_numpy()
X_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  'Height'].to_numpy()

# それぞれの平均
mu1 = np.mean(X_Human)
mu2 = np.mean(X_Hoge)
print(f'mu1 = {mu1:.2f}, mu2 = {mu2:.2f}')

次に，共通の分散 $\sigma^2$ を次のようにして推定します（ここでは標本分散を用いています）．

$$
\hat{\sigma}^2 = \frac{1}{\mbox{データの総数}}\left( \sum_{n:\mbox{人間}} (x_n - \mu_1)^2 + \sum_{n:\mbox{ほげ星人}} (x_n - \mu_2)^2 \right)
$$


In [ ]:
# (X_Human - mu1) と (X_Hoge - mu2) の2乗の平均
XX = np.hstack((X_Human - mu1, X_Hoge - mu2))
sigma2 = np.mean(XX**2)
print(f'sigma2 = {sigma2:.2f}')

身長のヒストグラムに ${\cal N}(\mu_1, \sigma^2)$ と ${\cal N}(\mu_2, \sigma^2)$ それぞれの確率密度関数を重ねて描くと，こんなんです．



In [ ]:
xmin, xmax = 80, 220
bins = np.linspace(xmin, xmax, 28)
xx = np.linspace(xmin, xmax, 100)

fig, ax = plt.subplots(figsize=(8, 6))
ax.hist(X_Human, bins=bins, alpha=0.8, label='Human', density=True)
ax.hist(X_Hoge,  bins=bins, alpha=0.8, label='Hoge',  density=True)

# 人間クラスの正規分布
px1 = norm.pdf(xx, loc=mu1, scale=np.sqrt(sigma2))
ax.plot(xx, px1, linewidth=2, color='blue')

# ほげ星人クラスの正規分布
px2 = norm.pdf(xx, loc=mu2, scale=np.sqrt(sigma2))
ax.plot(xx, px2, linewidth=2, color='red')

# 判別境界
ax.axvline((mu1+mu2)/2, color='purple', label=f'z(x) = 0')

ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.legend()
plt.show()

紫の直線は，判別関数 $z(x)$ の値が $0$ となる $x$ の値を表します．$z(x)$ によって人間クラスとほげ星人クラスを判別するときに，この点がちょうど2クラスの境界となっています．式$(7)$ の下に書いた通り，$z(x) = 0 \Leftrightarrow x = \frac{\mu_1+\mu_2}{2}$ です．

In [ ]:
print(f'(mu1+mu2)/2 = {(mu1+mu2)/2:.2f}')

したがって，

- 身長が 160cm のひとは，$z(160) > 0$ なので，人間と予測される
- 身長が 140cm のひとは，$z(140) < 0$ なので，ほげ星人と予測される

ことになります．


与えられたデータのそれぞれについて判別関数の値を計算し，予測が正しいかどうか調べてみましょう．

In [ ]:
# Z_Human[n] は X_Human[n] の判別関数の値
Z_Human = (mu1 - mu2)/sigma2*(X_Human - (mu1 + mu2)/2)
# Z_Hoge[n] は X_Hoge[n] の判別関数の値
Z_Hoge  = (mu1 - mu2)/sigma2*(X_Hoge  - (mu1 + mu2)/2)

cntHuman = np.sum(Z_Human >= 0)
cntHoge  = np.sum(Z_Hoge < 0)
print(f'人間:    正解数/データ数 = {cntHuman}/{len(X_Human)}')
print(f'ほげ星人: 正解数/データ数 = {cntHoge}/{len(X_Hoge)}')

人間については100人中96人を正しく人間と判別し（残り4人はほげ星人と判別した），ほげ星人については100人全員を正しくほげ星人と判別しています．
上のヒストグラムと密度関数のグラフからもおおよそ予想できる結果となっていますね．

次に，体重の方のデータで同じことをやってみましょう．

In [ ]:
# X_Human が「人間」の体重，X_Hoge が「ほげ星人」の体重
X_Human = dfHoge.loc[dfHoge['Class'] == 'Human', 'Weight'].to_numpy()
X_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  'Weight'].to_numpy()

# それぞれの平均
mu1 = np.mean(X_Human)
mu2 = np.mean(X_Hoge)
print(f'mu1 = {mu1:.2f}, mu2 = {mu2:.2f}')

# (X_Human - mu1) と (X_Hoge - mu2) の2乗の平均
XX = np.hstack((X_Human - mu1, X_Hoge - mu2))
sigma2 = np.mean(XX**2)
print(f'sigma2 = {sigma2:.2f}')

# 判別境界
print(f'(mu1+mu2)/2 = {(mu1+mu2)/2:.2f}')

In [ ]:
xmin, xmax = 0, 150
bins = np.linspace(xmin, xmax, 28)
xx = np.linspace(xmin, xmax, 100)

fig, ax = plt.subplots(figsize=(8, 6))
ax.hist(X_Human, bins=bins, alpha=0.8, label='Human', density=True)
ax.hist(X_Hoge,  bins=bins, alpha=0.8, label='Hoge',  density=True)

# 人間クラスの正規分布
px1 = norm.pdf(xx, loc=mu1, scale=np.sqrt(sigma2))
ax.plot(xx, px1, linewidth=2, color='blue')

# ほげ星人クラスの正規分布
px2 = norm.pdf(xx, loc=mu2, scale=np.sqrt(sigma2))
ax.plot(xx, px2, linewidth=2, color='red')

# 判別境界
ax.axvline((mu1+mu2)/2, color='purple', label=f'z(x) = 0')

ax.set_xlim(xmin, xmax)
ax.set_xlabel('Weight')
ax.legend()
plt.show()

クラスを予測させてみると...




In [ ]:
# Z_Human[n] は X_Human[n] の判別関数の値
Z_Human = (mu1 - mu2)/sigma2*(X_Human - (mu1 + mu2)/2)
# Z_Hoge[n] は X_Hoge[n] の判別関数の値
Z_Hoge  = (mu1 - mu2)/sigma2*(X_Hoge  - (mu1 + mu2)/2)

cntHuman = np.sum(Z_Human >= 0)
cntHoge  = np.sum(Z_Hoge < 0)
print(f'人間:    正解数/データ数 = {cntHuman}/{len(X_Human)}')
print(f'ほげ星人: 正解数/データ数 = {cntHoge}/{len(X_Hoge)}')

体重を変数とした場合の判別の境界は約 81 kgで，値がこれ以下なら人間，これより大きければほげ星人と予測される結果となりました．
ヒストグラムと密度関数のグラフから明らかなように，人間とほげ星人の値が入り混じっていますので，あまりよい予測ができていません．


---
### 2クラス多次元の判別分析

前のセクションでは，判別分析の方法を，具体的な計算の仕方も含めて説明するために，最も簡単な2クラス1次元の場合に限定して考えていました．(身長,体重)という2つの変数を持つデータを例にしてはいましたが，どちらかの変数だけで判別分析を行っていました．ここでは，2つ以上の変数を持つ多次元データの判別分析について説明し（クラス数は2に限定したまま），両方の変数を使って判別分析する実験を行ってみます．

#### 考え方



多次元データの場合も考え方は同じです．$D$ 次元のデータを「クラス1」と「クラス2」の2クラスに分類する問題の場合，

- クラス1の正規分布 ${\cal N}\left(\mathbf{\mu}_1, \Sigma_1\right)$
- クラス2の正規分布 ${\cal N}\left(\mathbf{\mu}_2, \Sigma_2\right)$

という2つを考えます．これらの正規分布のパラメータについては，与えられたデータから適当な方法で推定します．

このとき，「$\mathbf{x}$ という値をもつデータがクラス1の正規分布から得られたとすることの尤度」 $\ell_1(\mathbf{x})$ と，「$\mathbf{x}$ という値をもつデータがクラス2の正規分布から得られたとすることの尤度」$\ell_2(\mathbf{x})$ はそれぞれ，次式で与えられます．

$$
\begin{aligned}
\ell_1(\mathbf{x}) &= \frac{1}{\sqrt{(2\pi)^D|\Sigma_1|}} \exp{\left( -\frac{1}{2} (\mathbf{x}-\mathbf{\mu}_1)^{\top}\Sigma_1^{-1}(\mathbf{x}-\mathbf{\mu}_1) \right)} & (8)\\
\ell_2(\mathbf{x}) &= \frac{1}{\sqrt{(2\pi)^D|\Sigma_2|}} \exp{\left( -\frac{1}{2} (\mathbf{x}-\mathbf{\mu}_2)^{\top}\Sigma_2^{-1}(\mathbf{x}-\mathbf{\mu}_2) \right)} & (9)\\
\end{aligned}
$$

クラスが未知のデータ $\mathbf{x}$ のクラスを判別したいときは，$\ell_1(\mathbf{x}), \ell_2(\mathbf{x})$ の値を計算して，$\ell_1(\mathbf{x}) \geq \ell_2(\mathbf{x})$ なら $\mathbf{x}$ はクラス1に属するとし，さもなくばクラス2に属するとします．
しかし実際には，計算を簡単にするため，尤度の代わりにその対数をとった対数尤度を使うことがよくあります．

いまの場合，対数尤度は次のようになります．

$$
\begin{aligned}
\log{\ell_1(\mathbf{x})} &= -\frac{D}{2}\log(2\pi) - \frac{1}{2}\log{|\Sigma_1|} -\frac{1}{2} (\mathbf{x}-\mathbf{\mu}_1)^{\top}\Sigma_1^{-1}(\mathbf{x}-\mathbf{\mu}_1)  & (10)\\
\log{\ell_2(\mathbf{x})} &= -\frac{D}{2}\log(2\pi) - \frac{1}{2}\log{|\Sigma_2|} - \frac{1}{2} (\mathbf{x}-\mathbf{\mu}_2)^{\top}\Sigma_2^{-1}(\mathbf{x}-\mathbf{\mu}_2) & (11)
\end{aligned}
$$

$\log x$ は単調増加関数ですので，
$\log{\ell_1(\mathbf{x})} \geq \log{\ell_2(\mathbf{x})}$ なら $\mathbf{x}$ はクラス1に属するとし，さもなくばクラス2に属するとする，ということになります．
式$(9),(10)$の第1項 $-\frac{D}{2}\log(2\pi)$ は両者で共通ですので，値の大小を比較するためだけなら無視しても構いません．


#### マハラノビス距離と線型判別関数

1次元の場合と同様に，多次元のデータの場合も，2クラスの正規分布に制約条件を加えることで，判別のための計算が（判別関数が）より簡単な式になります．具体的には，2クラスの正規分布は分散共分散行列が共通で，平均のみが異なると仮定します．

$\Sigma_1 = \Sigma_2 = \Sigma$ とおくと，式$(9),(10)$ の第1項に加えて第2項も両者で共通になりますので，2つの対数尤度の大小を比較するかわりに，次の2つの量の大小を比較すればよいことになります．

$$
\begin{aligned}
D_1^2(\mathbf{x}) &= (\mathbf{x}-\mathbf{\mu}_1)^{\top}\Sigma^{-1}(\mathbf{x}-\mathbf{\mu}_1) & (12)\\
D_2^2(\mathbf{x}) &= (\mathbf{x}-\mathbf{\mu}_2)^{\top}\Sigma^{-1}(\mathbf{\mathbf{x}}-\mathbf{\mu}_2)  & (13)
\end{aligned}
$$

ただし，対数尤度の式にあった負号がなくなっていますので，大小関係は逆になります．$D_1^2(\mathbf{x}) \leq D_2^2(\mathbf{x})$ ならクラス1，さもなくばクラス2，です．

1次元のときと同様に，式$(12),(13)$ に出てくる

$$
D(\mathbf{x}) = \sqrt{(\mathbf{x}-\mathbf{\mu})^{\top}\Sigma^{-1}(\mathbf{x}-\mathbf{\mu}) } \qquad (14)
$$

という量のことを，**マハラノビス距離** (Mahalanobis' distance) といいます（式$(12),(13)$はマハラノビス距離の2乗）．これは，「$\mathbf{x}$ から ${\cal N}(\mathbf{\mu},\Sigma)$ までの距離」を表す量と考えることができます．

例えば，下図のような2次元正規分布があったとしましょう．青い★が平均 $\mathbf{\mu}$ です．白い円は，$\mathbf{\mu}$ とのユークリッド距離がある一定値をとる点の集合です．一方，色付きの楕円（の輪郭線）は，$\mathbf{\mu}$ とのマハラノビス距離が一定値をとる点の集合です（前回，手計算で楕円になることを確認してましたね）．ユークリッド距離で測ると点 $A$ と $\mathbf{\mu}$ の距離は $B$ と $\mathbf{\mu}$ の距離と等しいですが，マハラノビス距離で測ると点AよりBの方が$\mathbf{\mu}$ に近いことになります．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Mahalanobis.png">

このように，マハラノビス距離は，扱っている正規分布の広がり方（データの散らばり方）を考慮した測り方をするものとなっています．

1次元の場合の線形判別関数と同様に，式 $(12),(13)$ を用いて次式で定義される関数 $z(\mathbf{x})$ を，**線形判別関数** といいます：

$$
z(\mathbf{x}) = \frac{D_2^2(\mathbf{x}) - D_1^2(\mathbf{x})}{2} \qquad (15)
$$

あるデータ $\mathbf{x}$ について $z(\mathbf{x}) \ge 0$ ならばこのデータはクラス1に属すると判定し，$z(\mathbf{x}) < 0$ ならばクラス2に属すると判定します．

以下に示すように，この式は，変形していくと $\mathbf{x}$ の一次式となっています．

$$
\begin{aligned}
z(\mathbf{x}) &= \frac{1}{2}\left( 
(\mathbf{x}-\mathbf{\mu}_2)^{\top}\Sigma^{-1}(\mathbf{x}-\mathbf{\mu}_2) - (\mathbf{x}-\mathbf{\mu}_1)^{\top}\Sigma^{-1}(\mathbf{\mathbf{x}}-\mathbf{\mu}_1)  \right)\\
&= \frac{1}{2}\left( \mathbf{x}^{\top}\Sigma^{-1}\mathbf{x} - \mathbf{x}^{\top}\Sigma^{-1}\mathbf{\mu}_2 - \mathbf{\mu}_2^{\top}\Sigma^{-1}\mathbf{x} + \mathbf{\mu}_2^{\top}\Sigma^{-1}\mathbf{\mu}_2 -\left( \mathbf{x}^{\top}\Sigma^{-1}\mathbf{x} - \mathbf{x}^{\top}\Sigma^{-1}\mathbf{\mu}_1 - \mathbf{\mu}_1^{\top}\Sigma^{-1}\mathbf{x} + \mathbf{\mu}_1^{\top}\Sigma^{-1}\mathbf{\mu}_1\right) \right)\\
&= \frac{1}{2}\left( -2\mathbf{\mu}_2^{\top}\Sigma^{-1}\mathbf{x} + \mathbf{\mu}_2^{\top}\Sigma^{-1}\mathbf{\mu}_2 + 2\mathbf{\mu}_1^{\top}\Sigma^{-1}\mathbf{x} - \mathbf{\mu}_1^{\top}\Sigma^{-1}\mathbf{\mu}_1 \right)\\
&= (\mathbf{\mu}_1 - \mathbf{\mu}_2)^{\top}\Sigma^{-1}\mathbf{x} - \frac{1}{2}(\mathbf{\mu}_1 - \mathbf{\mu}_2)^{\top}\Sigma^{-1}(\mathbf{\mu}_1+\mathbf{\mu}_2)\\
&= (\mathbf{\mu}_1 - \mathbf{\mu}_2)^{\top}\Sigma^{-1}\left( \mathbf{x} - \frac{\mathbf{\mu}_1+\mathbf{\mu}_2}{2} \right) \qquad (16)
\end{aligned}
$$

$z(\mathbf{x}) = 0 \Leftrightarrow \mathbf{x} = \frac{\mathbf{\mu}_1+\mathbf{\mu}_2}{2}$ ですので，クラス1とクラス2を分ける境界は，2つの平均の中点を通り，ベクトル $\Sigma^{-1}(\mathbf{\mu}_1 - \mathbf{\mu}_2)$ に垂直な $(D-1)$次元平面（$D=2$のときは直線）となります（注）．

※注意: 高校のベクトルの項で学ぶ直線の方程式の話の3次元以上への一般化．$\mathbf{n}^{\top}(\mathbf{x}-\mathbf{p}) = \mathbf{n}\cdot (\mathbf{x}-\mathbf{p}) = 0$ を満たす点 $\mathbf{x}$ は，点 $\mathbf{p}$ を通り $\mathbf{n}$ を法線ベクトルとする平面上にあります．


#### 「人間」vs「ほげ星人」の判別分析

「人間」vs「ほげ星人」の $(身長, 体重)$ データを判別分析してみましょう．



In [ ]:
# データを格納した2次元配列をつくる
X_Human = dfHoge.loc[dfHoge['Class'] == 'Human', ['Height', 'Weight']].to_numpy()
X_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  ['Height', 'Weight']].to_numpy()

print('人間の最初の5人')
print(X_Human[:5, :])
print()
print('ほげ星人の最初の5人')
print(X_Hoge[:5, :])

まず，人間の平均 `mu_Human` とほげ星人の平均 `mu_Hoge` を求めます．


In [ ]:
# 人間クラスとほげ星人クラスそれぞれの平均
mu_Human = np.mean(X_Human, axis=0)
mu_Hoge  = np.mean(X_Hoge,  axis=0)
print('mu_Human = ', mu_Human)
print('mu_Hoge  = ', mu_Hoge)

次に，共通の分散共分散行列 `cov` を求めます．

In [ ]:
# 平均を引いたデータ行列をつくる
XX = np.vstack((X_Human - mu_Human, X_Hoge - mu_Hoge))
print(XX.shape)

# それを用いて共通の分散共分散行列を求める
cov = XX.T @ XX / len(XX)
print(cov)

判別分析の結果を直感的に理解できるように，データの散布図に重ねて，人間クラスほげ星人クラスそれぞれの推定された正規分布の等高線を描いてみましょう．

In [ ]:
##### 判別分析の結果を可視化する
#
fig, ax = plt.subplots(figsize=(12, 6))
xmin, xmax = 0, 250
ymin, ymax = 0, 150
xx, yy = np.mgrid[xmin:xmax:0.1, xmin:xmax:0.1]

# 人間クラスの散布図と推定された正規分布の等高線
ax.scatter(X_Human[:, 0], X_Human[:, 1], label='Human')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu_Human, cov=cov)
ax.contour(xx, yy, zz, colors='blue')

# ほげ星人クラスの散布図と推定された正規分布の等高線
ax.scatter(X_Hoge[:, 0], X_Hoge[:, 1], label='Hoge')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu_Hoge, cov=cov)
ax.contour(xx, yy, zz, colors='red')

# 判別境界
w = np.linalg.inv(cov) @ (mu_Human - mu_Hoge)
mu_mu = (mu_Human + mu_Hoge)/2
xp0, yp0 = w @ mu_mu / w[0], 0
xp1, yp1 = w @ (mu_mu - [0, ymax]) / w[0], ymax
ax.plot([xp0, xp1], [yp0, yp1], color='purple', label=f'z(x)=0')

ax.set_xlim(xmin, xmax)
ax.set_xlabel('Height')
ax.set_ylim(ymin, ymax)
ax.set_ylabel('Weight')
ax.set_aspect('equal')
ax.legend()
plt.show()

この図に描かれた紫色の直線は，判別分析によって得られた2クラスの境界，すなわち，判別関数 $z(\mathbf{x}) = 0$ となる $\mathbf{x}$ の値の集合（直線）です．
$(\mbox{身長},\mbox{体重}) = (150, 60)$ のひとは人間と判別され，$(150, 100)$ のひとはほげ星人と判別されることがわかります．

式$(16)$の判別関数の値を実際に計算して，上記の通りになることを確認してみましょう（注）．

※注: ここでは，簡単のため，式$(16)$の通りに分散共分散行列の逆行列を求めて判別関数の値を計算しています．しかし，実用的には，分散共分散行列の固有値分解を利用して式$(16)$を変形し，逆行列を使わずに済むようした方が効率的です（
$\Sigma = U\Lambda U^{\top}$ のとき，$\Sigma^{-1} = U\Lambda^{-1}U^{\top}$．$\Lambda$ は固有値を並べた対角行列，$U$ は固有ベクトルを並べた行列．$\Lambda$ は対角行列なので，$\Sigma^{-1}$ よりも $\Lambda^{-1}$ の方がずっと簡単に求まる）．



In [ ]:
# 判別したいデータ
XX = np.array([[150,  60],
               [150, 100],
               [180, 145],
               [180, 146]])

# cov の逆行列
covI = np.linalg.inv(cov)

# 判別関数の値を求める
ZZ = (XX - (mu_Human + mu_Hoge)/2) @ covI @ (mu_Human - mu_Hoge)

# 結果の表示
for n in range(len(XX)):
    print(f'値 {XX[n, :]} のひとの判別関数の値は {ZZ[n]: .2f} なので', end='')
    if ZZ[n] >= 0:
        s = '人間'
    else:
        s = 'ほげ星人'
    print(f' {s} と判別されます')

元のデータの判別結果がどうなるかも計算してみましょう．

In [ ]:
# Z_Human[n] は X_Human[n] の判別関数の値
Z_Human = (X_Human - (mu_Human + mu_Hoge)/2) @ covI @ (mu_Human - mu_Hoge)
# Z_Hoge[n] は X_Hoge[n] の判別関数の値
Z_Hoge  = (X_Hoge  - (mu_Human + mu_Hoge)/2) @ covI @ (mu_Human - mu_Hoge)

cntHuman = np.sum(Z_Human >= 0)
cntHoge  = np.sum(Z_Hoge < 0)
print(f'人間:    正解数/データ数 = {cntHuman}/{len(X_Human)}')
print(f'ほげ星人: 正解数/データ数 = {cntHoge}/{len(X_Hoge)}')

身長と体重という2つの値を組み合わせて予測できるため，こちらの方が良い結果が得られたようです．